In [ ]:
import pandas as pd


In [16]:
#import all flat files

INF_Feature = pd.read_excel (r'/Users/joezhou/Downloads/ALL_INFO_FEATURES.xlsx') 
PRICE_Feature = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Prices_features.xlsx')
FIN_Feature = pd.read_excel (r'/Users/joezhou/Downloads/ALL_FIN_features.xlsx')
DIV_Feature = pd.read_excel (r'/Users/joezhou/Downloads/ALL_Div_FEATURES.xlsx') 
 


In [36]:
# Find data coverage

import pandas as pd

TIC = pd.read_csv(r'/Users/joezhou/Downloads/ALL_TICKER.csv')
TIC_U = TIC.drop_duplicates(subset=['colummn']).rename(columns={'colummn': 'TickName'})[['TickName']]
TIC_U['Counter'] = 1


INF_U = INF_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
INF_U['IN_INFO']="In Info Extract"

PRICE_U = PRICE_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
PRICE_U['IN_PRICE']="In Price Extract"

FIN_U = FIN_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
FIN_U['IN_FIN']="In Financial Extract"

DIV_U = DIV_Feature.drop_duplicates(subset=['TickName'])[['TickName']]
DIV_U['IN_DIV']="In Dividend Extract"


In [37]:
# Join it back together
TIC_COVERAGE = TIC_U.merge(INF_U,how='left',on='TickName').merge(PRICE_U,how='left',on='TickName').merge(FIN_U,how='left',on='TickName').merge(DIV_U,how='left',on='TickName').fillna('Missing')


Tbl_Coverage = TIC_COVERAGE.groupby(['IN_INFO','IN_PRICE','IN_FIN', 'IN_DIV'])['Counter'].count().reset_index().sort_values(by=['Counter'],ascending=False)
Tbl_Coverage['Percentage'] = 100 * Tbl_Coverage['Counter']  / Tbl_Coverage['Counter'].sum()

Tbl_Coverage.head(27)
# To-do: add market cap data in here to quantify the proportion missing

,IN_INFO,IN_PRICE,IN_FIN,IN_DIV,Counter,Percentage
3,In Info Extract,In Price Extract,Missing,Missing,806,37.558248
2,In Info Extract,In Price Extract,Missing,In Dividend Extract,501,23.345760
6,In Info Extract,Missing,Missing,Missing,416,19.384902
5,In Info Extract,Missing,Missing,In Dividend Extract,230,10.717614
9,Missing,Missing,Missing,Missing,180,8.387698
8,Missing,In Price Extract,Missing,Missing,7,0.326188
0,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,2,0.093197
7,Missing,In Price Extract,Missing,In Dividend Extract,2,0.093197
1,In Info Extract,In Price Extract,In Financial Extract,Missing,1,0.046598
4,In Info Extract,Missing,In Financial Extract,Missing,1,0.046598


In [39]:

#Remove tickers without complete variable set
dp_cov_comp = TIC_COVERAGE[(TIC_COVERAGE.IN_INFO == 'In Info Extract') 
                    & (TIC_COVERAGE.IN_PRICE == 'In Price Extract')
                    & (TIC_COVERAGE.IN_FIN == 'In Financial Extract')
                    & (TIC_COVERAGE.IN_DIV == 'In Dividend Extract')
                    ]




In [40]:
dp_cov_comp.count()

TickName    2
Counter     2
IN_INFO     2
IN_PRICE    2
IN_FIN      2
IN_DIV      2
dtype: int64

In [41]:

# Combine the feature sets

TIC_FLAT = dp_cov_comp.merge(PRICE_Feature,how='left',on='TickName').merge(DIV_Feature,how='left',on='TickName').merge(INF_Feature,how='left',on='TickName')

# .merge(FIN_Feature,how='left',on='TickName')


# .fillna('Missing')

# merge(INF_Feature,how='left',on='TickName').


In [42]:
TIC_FLAT.head(5)

,TickName,Counter,IN_INFO,IN_PRICE,IN_FIN,IN_DIV,Unnamed: 0_x,absmvmtOPEN_MEDIAN,absmvmtOPEN_STD,Unnamed: 0_y,...,fullTimeEmployees,forwardEps,forwardPE,shortName,longName,longBusinessSummary,website,messageBoardId,industry,sector
0,ANZ.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,220,0.018436,0.007378,86,...,37506.0,2.47,10.238867,ANZ BANK FPO,Australia and New Zealand Banking Group Limited,Australia and New Zealand Banking Group Limite...,http://www.anz.com.au,finmb_384998,Banks—Diversified,Financial Services
1,CBA.AX,1,In Info Extract,In Price Extract,In Financial Extract,In Dividend Extract,476,0.014674,0.007270,194,...,41778.0,5.54,16.000000,CWLTH BANK FPO,Commonwealth Bank of Australia,Commonwealth Bank of Australia provides integr...,http://www.commbank.com.au,finmb_874262,Banks—Diversified,Financial Services


In [43]:
TIC_FLAT.to_excel('/Users/joezhou/Downloads/TIC_FLAT.xlsx')  
